In [33]:
from humanitarian_management_system.models.resourceAllocator import ResourceAllocator
from humanitarian_management_system.models.resourceAdder import ResourceAdder
from humanitarian_management_system.models.resourceReport import ResourceReport

from pathlib import Path
import pandas as pd

In [34]:
df = pd.read_csv("../data/camp.csv")
print(df)

    campID  eventID countryID  refugeeCapacity healthRisk  volunteerPop  \
0        1        3        IN            10000       high             0   
1        2        7        CN               15        low             0   
2        3        7        CN                8       high             0   
3        4        4        IN               20       high             0   
4        5        3        IN               12        low             0   
5        6        7        CN               13        low             2   
6        7        4        IN               15       high             0   
7        8        8        EG               15       high             1   
8        9        8        EG                7       high             1   
9       11        4        IN               10       high             0   
10      12        3        IN              111        low             0   
11      13        5        IN               12       high             0   
12      14        3      

In [35]:
condition = (df['refugeePop']>0) & (df['status']=='open')
valid_open = df.loc[condition, ['campID', 'refugeePop']]
print(valid_open['campID'])

2     3
4     5
8     9
9    11
Name: campID, dtype: int64


In [36]:
resource_stock_csv_path = "../data/resourceStock.csv"
resource_allocation_csv_path = "../data/resourceAllocation.csv"
resource_unallocated_stock_csv_path = "../data/resourceUnallocatedStock.csv"

totalResources_df = pd.read_csv(resource_stock_csv_path)
resourceAllocs_df = pd.read_csv(resource_allocation_csv_path)
unallocResources_df = pd.read_csv(resource_unallocated_stock_csv_path)
joined_df = pd.merge(totalResources_df, resourceAllocs_df, on='resourceID', how='inner') # does not factor in the camps that dont have allocation but have non zero refugees 


In [37]:
print(joined_df)
camps_in_resource_map = joined_df['campID'].unique()

    resourceID                      name  total  priorityLvl  campID    qty
0            1                  Blankets  10084            2       3   3601
1            1                  Blankets  10084            2       5   2881
2            1                  Blankets  10084            2       6   1441
3            1                  Blankets  10084            2       9   1441
4            1                  Blankets  10084            2      11    720
5            2                  Clothing   2850            1       3   1018
6            2                  Clothing   2850            1       5    814
7            2                  Clothing   2850            1       6    407
8            2                  Clothing   2850            1       9    407
9            2                  Clothing   2850            1      11    204
10           3                      Toys     65            3       3     23
11           3                      Toys     65            3       5     19
12          

In [38]:
resource_camp = joined_df.pivot_table(index=['resourceID', 'name', 'priorityLvl'], columns='campID', values='qty', aggfunc='sum').sort_index(level=0)

resource_camp.reset_index(inplace=True)



In [39]:
resource_c_id = resource_camp[['resourceID',3]]

        # need to recalculate all 3 sheets 

        # first calculate...
        # unallocated stock - add the above to each resource. do mapping to be on safe side 
inventory = unallocResources_df ## will always have more or equal 
joined_df = pd.merge(inventory, resource_c_id, on='resourceID', how='outer').fillna(0) #join and then add... 


joined_df['unallocTotal'] = joined_df[3] + joined_df['unallocTotal']
joined_df.drop(3, axis=1, inplace=True)

In [40]:
print(joined_df)

    resourceID                      name  unallocTotal  priorityLvl
0            1                  Blankets          3601            2
1            2                  Clothing          1018            1
2            3                      Toys            23            3
3            4                  Medicine            36            1
4            5                  Vitamins            30            2
5            6                      Food            36            1
6            7                    Snacks            23            3
7            8                Toiletries            30            2
8            9                     Water            40            1
9           10             Baby Supplies            36            1
10          11  General Tools & Supplies         17891            3


In [57]:
map = resourceAllocs_df
remove = map['campID'] == 3
new_map = map[~remove]

In [58]:
assigned = totalResources_df
joined_df = pd.merge(assigned.drop(['total'], axis=1, inplace=False), new_map, on='resourceID', how='inner')
resource_sum = joined_df.groupby('resourceID').agg({
                'name': 'first',  # Keeps the first name for each group
                'qty': 'sum',  # Sums the qty for each camp >> we ignore the 'total' column in stockManualResources
                'priorityLvl': 'first',  
            }).reset_index()
resource_sum.rename(columns={'qty': 'total'}, inplace=True)

print(resource_sum)

    resourceID                      name  total  priorityLvl
0            1                  Blankets   6483            2
1            2                  Clothing   1832            1
2            3                      Toys     42            3
3            4                  Medicine     66            1
4            5                  Vitamins     54            2
5            6                      Food     66            1
6            7                    Snacks     42            3
7            8                Toiletries     54            2
8            9                     Water     72            1
9           10             Baby Supplies     66            1
10          11  General Tools & Supplies  32205            3


In [60]:
report = pd.merge(totalResources_df, unallocResources_df[['resourceID', 'unallocTotal']], on='resourceID', how='outer').fillna(0)
print(report)

    resourceID                      name  total  priorityLvl  unallocTotal
0            1                  Blankets  10084            2             0
1            2                  Clothing   2850            1             0
2            3                      Toys     65            3             0
3            4                  Medicine    102            1             0
4            5                  Vitamins     54            2            30
5            6                      Food    102            1             0
6            7                    Snacks     65            3             0
7            8                Toiletries     84            2             0
8            9                     Water    112            1             0
9           10             Baby Supplies    102            1             0
10          11  General Tools & Supplies  50096            3             0


In [64]:
df = pd.DataFrame(map['resourceID'].unique())
df.rename(columns={0: 'resourceID'}, inplace=True)
df['campID'] = 100
df['qty'] = 50
print(df)

    resourceID  campID  qty
0            1     100   50
1            2     100   50
2            3     100   50
3            4     100   50
4            5     100   50
5            6     100   50
6            7     100   50
7            8     100   50
8            9     100   50
9           10     100   50
10          11     100   50
